In [2]:
## Bajada de datos guantes de nitrilo en argentina Mercadolibre

import requests
import pandas as pd
import time
import csv

# Definimos la URL base de la API y el término de búsqueda
url_base = "https://api.mercadolibre.com/sites/MLA/search"
query = "guantes de nitrilo"

# Parámetros iniciales de la consulta
limit = 50
offset = 0

# Lista para almacenar los diccionarios de cada ítem
items_list = []

# Claves relevantes que queremos extraer
keys_to_extract = [
    'id', 'title', 'condition', 'price', 'currency_id', 'available_quantity', 'sold_quantity',
    'buying_mode', 'listing_type_id', 'stop_time', 'permalink', 'thumbnail', 'category_id', 'domain_id'
]

# Función para limpiar valores
def clean_value(value):
    if isinstance(value, str):
        for char in ['"', ',', '\n', '\r']:
            value = value.replace(char, '')
        return value
    return value

while True:
    # Parámetros para la llamada actual
    params = {
        "q": query,
        "limit": limit,
        "offset": offset
    }
    
    response = requests.get(url_base, params=params)
    data = response.json()
    
    # Verificamos si la consulta fue exitosa
    if response.status_code != 200:
        print("Error al obtener los datos de la API")
        break
    
    # Obtenemos los items
    items = data.get('results', [])
    
    if not items:
        print("No hay más resultados")
        break

    for item in items:
        # Crear un diccionario con las claves específicas
        item_dict = {key: item.get(key) for key in keys_to_extract}

        # Limpiar los valores del diccionario de item
        for key, value in item_dict.items():
            item_dict[key] = clean_value(value)

        # Desanidar atributos
        if 'attributes' in item:
            for attribute in item['attributes']:
                attr_name = attribute.get('name')
                attr_value = attribute.get('value_name')
                if attr_name and attr_value:
                    # Limpiar nombre y valor de los atributos
                    attr_name = clean_value(attr_name)
                    attr_value = clean_value(attr_value)
                    item_dict[attr_name] = attr_value
        
        items_list.append(item_dict)
    
    # Incrementar el offset para la siguiente llamada
    offset += limit
    
    # Si hemos alcanzado el límite de 1000 resultados, detener el bucle
    if offset >= 1000:
        print("Se ha alcanzado el límite de 1000 resultados")
        break
    
# Crear un DataFrame de Pandas con la lista de diccionarios
df = pd.DataFrame(items_list)

# Mostrar las primeras filas del DataFrame
print(df.head())

# Guardar el DataFrame a un archivo CSV sin comillas dobles
df.to_csv("guantes_nitrilo_datos.csv", index=False, encoding='utf-8-sig')


Se ha alcanzado el límite de 1000 resultados
              id                                              title condition  \
0  MLA1426151013  Guantes De Nitrilo Descartables Antideslizante...       new   
1  MLA1781901876  Guantes Descartables Antideslizantes Selectsho...       new   
2  MLA1409032303  Guantes Descartables Antideslizantes One Glove...       new   
3  MLA1652591318  Caja Guantes Nitrilo Primera Calidad Con Anmat...       new   
4  MLA1787907006  Guantes Descartables Euromix Nitrilo Rosa X 10...       new   

     price currency_id  available_quantity sold_quantity buying_mode  \
0   6289.0         ARS                 500          None  buy_it_now   
1   6289.0         ARS                5000          None  buy_it_now   
2  59773.0         ARS                   1          None  buy_it_now   
3   6998.0         ARS                5000          None  buy_it_now   
4   6980.0         ARS                   1          None  buy_it_now   

  listing_type_id                 s